# CS316 Project

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("credit_analysis") \
    .getOrCreate()


bash: /home/huukhang1512/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by bash)
bash: /home/huukhang1512/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by bash)
23/05/29 16:06:26 WARN Utils: Your hostname, huukhang1512-B550I-AORUS-PRO-AX resolves to a loopback address: 127.0.1.1; using 192.168.0.162 instead (on interface wlp6s0)
23/05/29 16:06:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/29 16:06:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/29 16:06:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
df_original=spark.read.format("csv").option("header","true").load('./data.csv', inferSchema=True)

In [3]:
df_original.count()

855969

## We realised there's many rows that have more delimeters than the others, therefore, we decided to remove some record with wrong delimeter before hand

In [4]:
def remove_lines_with_excessive_commas(input_file, output_file):
    with open(input_file, 'r') as csv_input, open(output_file, 'w', newline='') as csv_output:
        first_line = csv_input.readline()
        csv_output.write(first_line)
        number_of_comma = first_line.count(',')
        lines_removed = 0

        for row in csv_input.readlines():
            
            if row.count(',') <= number_of_comma:
                csv_output.write(row)
            else:
                lines_removed += 1

        print(f"{lines_removed} Lines with more than {number_of_comma} commas have been removed.")

# Usage example
input_file = 'data.csv'
output_file = 'output.csv'

In [5]:
remove_lines_with_excessive_commas(input_file, output_file)

51895 Lines with more than 72 commas have been removed.


In [6]:
df=spark.read.format("csv").option("header","true").load('./output.csv', inferSchema=True)

In [7]:
df.count()

804074

In [8]:
df.show(5)

23/05/29 16:06:35 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------------+----------+--------------+----------+-------------------+----------+----------+--------------------+--------------+--------------------+--------+----------+-----+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------+

In [9]:
df.filter(df['default_ind'] == 1).count()

40660

In [10]:
df.columns

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'pymnt_plan',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 '

In [11]:
df.groupBy(df["default_ind"]).count().show()

+-----------+------+
|default_ind| count|
+-----------+------+
|          1| 40660|
|          0|763414|
+-----------+------+



5% of the data are `default`

Visualisation of default diagram

In [12]:
df.groupby(['default_ind']).count().to_pandas_on_spark(index_col='default_ind').plot.pie(y='count')

/home/huukhang1512/.local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


We will need to resample a training data that will have equal distribution (50/50) of default and non-default rows

In [14]:
df.groupby(['application_type']).count().to_pandas_on_spark(index_col=['application_type']).plot.pie(y='count')

The number of data with `application_type == 'JOINT'`  are heavily underrepresent

Checking if there's any default cases when `application_type == 'JOINT'`

In [15]:
df.filter((df['application_type'] == "JOINT") & (df['default_ind'] == '1')).count()

0

Base on the dataset, virutally no `default` happened when the `application_type == 'JOINT'`!

In [16]:
df_original.filter(df_original['application_type'] == "JOINT").count()

442

We only remove 1 `"JOINT"` account from cleaning the CSV dataset

## Observation on `null` data fields:

In [17]:
null_dict = {col : df.filter(df[col].isNull()).count() for col in df.columns}

In [18]:
null_dict

{'id': 0,
 'member_id': 0,
 'loan_amnt': 0,
 'funded_amnt': 0,
 'funded_amnt_inv': 0,
 'term': 0,
 'int_rate': 0,
 'installment': 0,
 'grade': 0,
 'sub_grade': 0,
 'emp_title': 47020,
 'emp_length': 0,
 'home_ownership': 0,
 'annual_inc': 0,
 'verification_status': 0,
 'issue_d': 0,
 'pymnt_plan': 0,
 'desc': 723334,
 'purpose': 0,
 'title': 26,
 'zip_code': 0,
 'addr_state': 0,
 'dti': 0,
 'delinq_2yrs': 0,
 'earliest_cr_line': 0,
 'inq_last_6mths': 0,
 'mths_since_last_delinq': 408833,
 'mths_since_last_record': 677326,
 'open_acc': 0,
 'pub_rec': 0,
 'revol_bal': 0,
 'revol_util': 401,
 'total_acc': 0,
 'initial_list_status': 0,
 'out_prncp': 0,
 'out_prncp_inv': 0,
 'total_pymnt': 0,
 'total_pymnt_inv': 0,
 'total_rec_prncp': 0,
 'total_rec_int': 0,
 'total_rec_late_fee': 0,
 'recoveries': 0,
 'collection_recovery_fee': 0,
 'last_pymnt_d': 8755,
 'last_pymnt_amnt': 0,
 'next_pymnt_d': 216332,
 'last_credit_pull_d': 47,
 'collections_12_mths_ex_med': 34,
 'mths_since_last_major_dero

- String columns such as `emp_title`, `desc`, `title` won't contribute much to the model training later on, therefore these columns will be dropped
- all `*_joint` columns (`annual_inc_joint`, `dti_joint`, `verification_status_joint`) have 803633 null values, equivalent to the number of `INDIVIDUAL` `application_type` rows -> Therefore, only the `application_type` of `JOINT` will have these columns filled
- `null` value for `mths_since_last_delinq` indicate that this borrower does not have any delinquencies (does not behind on payment) record.
- `null` value for `mths_since_last_record` indicate that this borrower does not have any public record (does not have any public records of bankruptcy and judgement information)
- `null` value for `mths_since_last_major_derog` indicate that this borrower does not have any worse rating in the last 90 days


In [25]:
df.filter(df['last_pymnt_d'].isNull()).show()

+-------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------------+----------+--------------+----------+-------------------+----------+----------+--------------------+------------------+--------------------+--------+----------+-----+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+---------

In [28]:
df.filter(df['last_pymnt_d'].isNull() & (df['default_ind'] == 1)).count()

496

=> Not all record without `last_pyment_d` resulting in `default` (only 5.6% resulting in `default`). It may indicate that this a new created loan, where the borrower just took this loan and yet to pay their first repayment. Therefore we will keep this column

In [37]:
df.groupBy(df['verification_status_joint'])\
    .count()\
    .filter(df["verification_status_joint"].isNotNull())\
    .show()

+-------------------------+-----+
|verification_status_joint|count|
+-------------------------+-----+
|                 Verified|  138|
|          Source Verified|   51|
|             Not Verified|  252|
+-------------------------+-----+



In [39]:
df.groupBy(df['verification_status_joint'])\
    .count()\
    .filter(df["verification_status_joint"].isNotNull())\
    .to_pandas_on_spark(index_col=['verification_status_joint'])\
    .plot\
    .pie(y='count')

In [32]:
df.groupBy(df['verification_status']).count().show()

+-------------------+------+
|verification_status| count|
+-------------------+------+
|           Verified|259501|
|    Source Verified|304870|
|       Not Verified|239703|
+-------------------+------+



In [31]:
df.groupBy(df['verification_status'])\
    .count()\
    .to_pandas_on_spark(index_col=['verification_status'])\
    .plot.pie(y='count')

=> Evenly distributed accross all `INDIVIDUAL` application_type accounts 

Let's see the relationship between the number of personal finance inquiries (`inq_fi`) and default rate

In [61]:
df.filter(df['inq_fi'].isNotNull() & (df['default_ind'] == 1)).count()

0

=> All the borrowers with a record of personal finance inquiries virtually not default. If this field is `null`, it indicate that the borrowers never inquiries any personal finance advice from this firm.

Next, let's see the relationship between the number of credit inquiries in the last 12 months (`inq_last_12m`) and default rate

In [64]:
df.filter(df['inq_last_12m'].isNotNull()).sort('inq_last_12m', ascending=False).show()

+--------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------------+----------+--------------+----------+-------------------+----------+----------+----+------------------+--------------------+--------+----------+-----+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------+-----------

In [66]:
df.filter(df['inq_last_12m'].isNotNull() & (df['default_ind'] == 1)).sort('inq_last_12m', ascending=False).count()

0

In real life, higher number of number of credit inquiries will result in lower result credit, making this loan has a higher chance of default (https://www.canstar.com.au/credit-score/do-credit-checks-affect-your-credit-score/). However it is the opposite with this dataset, as more credit inquiries result in no default rate! 

In [20]:
df.select('total_pymnt', 'total_rec_int', 'total_rec_late_fee', 'int_rate', 'total_rec_prncp', 'funded_amnt', 'term', 'issue_d', 'last_pymnt_d', 'default_ind').show(100)

+-----------+-------------+------------------+--------+---------------+-----------+----------+----------+------------+-----------+
|total_pymnt|total_rec_int|total_rec_late_fee|int_rate|total_rec_prncp|funded_amnt|      term|   issue_d|last_pymnt_d|default_ind|
+-----------+-------------+------------------+--------+---------------+-----------+----------+----------+------------+-----------+
|5861.071414|       861.07|               0.0|   10.65|         5000.0|       5000| 36 months|01-12-2011|  01-01-2015|          0|
|    1008.71|       435.17|               0.0|   15.27|         456.46|       2500| 60 months|01-12-2011|  01-04-2013|          1|
|3003.653644|       603.65|               0.0|   15.96|         2400.0|       2400| 36 months|01-12-2011|  01-06-2014|          0|
|5631.377753|       631.38|               0.0|     7.9|         5000.0|       5000| 36 months|01-12-2011|  01-01-2015|          0|
|3938.144334|       938.14|               0.0|   18.64|         3000.0|       3000|

In [ ]:
df.select('last_pymnt_d', 'funded_amnt', 'total_rec_prncp', 'default_ind', 'total_rec_late_fee', 'total_rec_int', 'int_rate', 'total_pymnt', 'issue_d').filter((df["last_pymnt_d"] != '01-01-2016') & (df['default_ind'] == 1) & ((df['funded_amnt'] == df['total_rec_prncp'])| (df['funded_amnt'] < df['total_rec_prncp'] -10))).show()


+------------+-----------+---------------+-----------+------------------+-------------+--------+-----------+----------+
|last_pymnt_d|funded_amnt|total_rec_prncp|default_ind|total_rec_late_fee|total_rec_int|int_rate|total_pymnt|   issue_d|
+------------+-----------+---------------+-----------+------------------+-------------+--------+-----------+----------+
|  01-06-2015|       8000|         8000.0|          1|               0.0|       587.29|   13.66|8587.289426|01-11-2014|
+------------+-----------+---------------+-----------+------------------+-------------+--------+-----------+----------+



There's a lot of null values for 

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("credit_analysis") \
    .getOrCreate()
df=spark.read.format("csv").option("header","true").load('./data.csv', inferSchema=True)
# read csv and infer schema

In [ ]:
df.select(df["verification_status_joint"]).distinct()

39

In [ ]:
df.filter(df["funded_amnt"] != df['funded_amnt_inv'] and df["issue_d"]).show()

+-------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------------+----------+--------------+----------+-------------------+----------+----------+--------------------+------------------+--------------------+--------+----------+-----+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+---------